# ММС. Лабораторная работа №3.

## Описание игры
В условиях игры из лабораторной работы  №2, к детерминированным стратегиям добавим **стохастические** стратегии и сравним их все между собой.

## Сравнение стратегий
Чтобы сравнить пару стратегий, где хотя бы одна из них стохастическая, недостаточно одной игры. Необходимо запустить большое число симуляций и оценить по результатам:
1. Среднее число очков
2. Медианное число очков
3. Модальное число очков
4. Дисперсию числа очков

## Стратегии
* Hank - 0 или 1 равновероятно
* Ivan - 0 с вероятностью $0.9$
* Jack - сначала 0. если опонент в последнем ходу выдал 0, то 0, а если 1, то с вероятностью $0.25$ прощает и выдает 0
* Kevin - сначала 0. повторяет последний ход опонента, но с вероятностью $0.25$ выдает противоположное
* Lucas - до начала игры генерирует случайно целое число в интервале от 1 до 50. каждые стлько ходов выдает 1, остальные ходы - 0 
* Max - генерирует целое число в интервале от 0 до 20 и играет 0 столько ходов. затем генирирует число и играет 1 столько ходов. и так далее
* Natan - придумайте свою **стохастическую статегий**


## Характеристики
Пусть мы получили результаты случайных независимых экспериментов $x = (x_1, \dots, x_n)$. Еще этот набор называют **выборкой**.

**Среднее значение**
Мерой центрального положения данных является среднее значение. Обычное арифметическое среднее
$$
\overline{x} = \frac{\sum_i^n x_i}{n}
$$

**Дисперсия**
Мерой разброса значений относительно среднего является дисперсия. Средний квадрат отклонения от среднего
$$
S^2 = \frac{\sum_i^n (x_i - \overline{x})^2}{n-1}
$$

**Медиана**
Аналогом среднего значения служит медиана.
Медианой выборки называют такое значение $med(x)$, что половина значений выборки меньше этого значения.
Чтобы вычислить медиану, необходимо отсортировать выборку и взять физически центральный элемент. Если размер выборки четное число - берут среднее между двумя центральными элементами.

**Мода**
Мода это самое часто встречающееся значение в выборке (то значение, вероятность получить которое максимальна). Но в нашем случае скорее всего все значения будут уникальны, то есть встречаться один раз. Поэтому разобъем множество значений на интервалы, число которых можно определить по формуле $k = \log_2(1 + n)$. Посчитаем сколько результатов из выбрки попадает в каждый интервал. И внутри интервала с наибольшей частотой попаданий вычислим среднее. Это и будет оценкой моды.

---

Используем систему игр, выполненую в лабораторной работе №2

In [206]:
import random as rnd
import numpy as np
import pandas as pd

#pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)  # or None

In [207]:
#do not trust always
def AlexMove(movesMy, movesOp):
    move = 1
    return move
# trust always
def BobMove(movesMy, movesOp):
    move = 0
    return move
#begin with trust, then copy op move
def ClaraMove(movesMy, movesOp):
    if not movesOp:
        move = 0
    else:
        move = movesOp[-1]
    return move
    
#begin with trust, then do oposite op move
def DenisMove(movesMy, movesOp):
    if not movesOp:
        move = 0
    else:
        move = 0 if (movesOp[-1]) else 1
    return move


def EmmaMove(movesMy, movesOp):
    return (1 if (len(movesMy)+1) % 20== 0 else 0)
    
def FridaMove(movesMy, movesOp):        
    return 1 if sum(movesOp)>0 else 0

def GeorgeMove(movesMy, movesOp):
    gap = 3
    trust_threshold = 5
    limit = 4
    #trusts for some time
    if len(movesMy) < trust_threshold:
        return 0
    # trying to fool
    if len(movesMy) == trust_threshold:
        return 1

    # than trusts again for some time
    if len(movesMy) <= trust_threshold + gap:
        return 0

    #if not punished, fools, if punished proceeds with respect, if was fooled to many times fooling till end of game
    if sum(movesOp[trust_threshold:trust_threshold + gap]) == 0:
        return 1
    elif sum(movesOp[:trust_threshold + gap]) > limit:
        return 1
    else:
        return movesOp[-1]


def HawkMove(movesMy, movesOp):
    return 1 if rnd.random() > 0.5 else 0

    
def IvanMove(movesMy, movesOp):
    return 1 if rnd.random() > 0.9 else 0

def JackMove(movesMy, movesOp):
    if len(movesMy)==0:
        return 0
    elif movesOp[-1] == 0:
        return 0
    else: 
        return 0 if rnd.random() > 0.75 else 1
def KevinMove(movesMy, movesOp):
    if len(movesMy)==0:
        return 0
    return movesOp[-1] if rnd.random()> 0.75 else ( 0 if movesOp[-1] == 1 else 1) 

LucasData = 1
def LucasMove(movesMy, movesOp):
    if len(movesMy)==0:
        global LucasData
        LucasData = rnd.randint(1, 51)
    if len(movesMy) < LucasData:
        return 1
    else:
        return 0 

MaxData = None
def MaxMove(movesMy, movesOp):
    global MaxData
    if len(movesMy)== 0:
        MaxData = {'move': 0,      'movesLeft': 0}            
    if MaxData['movesLeft'] == 0:
        MaxData['movesLeft'] = rnd.randint(0, 20)
        MaxData['move'] = 1- MaxData["move"]
    move = MaxData["move"]
    MaxData["movesLeft"] -= 1
    return move
    
        
    
NatanData = None
def NatanMove(movesMy, movesOp):
    global NatanData
    if len(movesMy)== 0:
        NatanData = {"prob": 0.0,      'movesLeft': 0}            
    if NatanData['movesLeft'] == 0:
        NatanData['movesLeft'] = 10
        NatanData['prob'] = rnd.random()
    move = 1 if NatanData["prob"]< rnd.random() else 0
    NatanData["movesLeft"] -= 1
    return move

In [208]:


def PlayGame(player1, player2, show = False):
    WinTable = [ [3, 0], [ 5, 1]]
    movesFirst = []
    movesSecond = []

    
    Scores_p1 = []   
    for x in range(200):
        move1 = int(player1(movesFirst, movesSecond))
        move2 = int(player2(movesSecond, movesFirst))
        movesFirst.append(move1)
        movesSecond.append(move2)
        if (show):
            print("game #", x)
            print("Trust" if not move1 else "Fool", "Trust" if not move2 else "Fool")
            print("+", WinTable[move1][move2], "   +", WinTable[move2][move1], "\n")
            
        Scores_p1.append(WinTable[move1][move2])
        #Score_p2 += WinTable[move2][move1]    
    return (Scores_p1)

Strat = {"Alex" : AlexMove, "Bob" :BobMove, "Clara": ClaraMove, "Denis" : DenisMove,
         "Emma" : EmmaMove, "Frida" : FridaMove , "George" : GeorgeMove, "Hawk" : HawkMove,
         "Ivan" : IvanMove, "Jack" :JackMove, "Kevin" : KevinMove, "Lucas" : LucasMove,  "Max" : MaxMove, "Natan" : NatanMove } 

Players = Strat.keys()#= ["Alex", "Bob", "Clara", "Denis", "Emma", "Frida", "George", "Hawk", "Ivan", "Jack"]


In [209]:

GameTables = []
ExpNumber = 100
for x in range(ExpNumber):
    prog = (x + 1) / ExpNumber
    filled = int(50 * prog)
    bar = '█' * filled + '-' * (50 - filled)
    print(f'\r[{bar}] {prog*100:.1f}%', end='')

    
    matr = [[]]
    for p1 in Players:
        line = []
        for p2 in Players:
            line.append(PlayGame(Strat[p1], Strat[p2]))
        matr.append(line)
    matr.pop(0) 
    GamesData = pd.DataFrame(matr, index=Players, columns=Players)         
    GameTable = pd.DataFrame(matr, index=Players, columns=Players)     
    GameTables.append(GameTable)
    


[██████████████████████████████████████████████████] 100.0%

In [210]:
print("\n")
print("line = player\ncolumn = opponent\nitem = how much player gets")
ScoresData = GameTables[0].map(sum)

#print(GameTables[0]["Alex"]["Max"])

print(ScoresData)

print("--------------------------------------------------------\n    sum of scores against all strategies")
ScoresDataSum = ScoresData.sum(axis=1)

print(ScoresDataSum)



line = player
column = opponent
item = how much player gets
        Alex   Bob  Clara  Denis  Emma  Frida  George  Hawk  Ivan  Jack  Kevin  Lucas  Max  Natan
Alex     200  1000    204   1000   960    204     232   656   916   412    816    932  572    548
Bob        0   600    600      3   570    600      24   267   549   600    114    576  267    354
Clara    199   600    600    450   588    600     599   464   579   600    444    539  529    423
Denis      0   998    450    400   940      9      38   423   874   542    463    743    0    389
Emma      10   620    593     40   580     71      34   307   523   602    164    511  398    334
Frida    199   600    600    989   921    600     221   607   917   600    821    943  855    645
George   192   984    599    958   944    211     598   567   884   420    825    808  260    632
Hawk     112   808    435    472   774     98     131   427   719   527    445    729  715    525
Ivan      19   650    574     85   603     33     574   

In [211]:
import math

def CalcAvg(GameTables):
    TableSum = GameTables[0].map(sum)
    for tableIdx in range(1, len(GameTables)):
        TableSum+=GameTables[tableIdx].map(sum)
    return TableSum.map(lambda x: x/len(GameTables))

def CalcDisp(GameTables, TabelAvg):    
    TableDisp = GameTables[0].map(sum)
    TableDisp.iloc[:] = 0
    for tableIdx in range(0, len(GameTables)):
        TableDisp+= (GameTables[tableIdx].map(sum) - TableAvg).map(lambda x: x*x)
    print(len(GameTables))
    TableDisp = TableDisp.map(lambda x : math.sqrt(x/(len(GameTables)-1)))
    return TableDisp

def CalcМеdian(GameTables):
    TableMed = GameTables[0].map(sum)
    TableMed.iloc[:] = 0
    TableScores = GameTables[0].map(lambda x: [sum(x)])
    #print(TableScores)
    for Table in GameTables:
        TableScores += Table.map(lambda x: [sum(x)])
    TableMed = TableScores.map(lambda x: x[int(len(GameTables)/2)])
    return TableMed

def CalcMode(GameTables):
    TableScores = GameTables[0].map(lambda x: [sum(x)])
    for tableIdx in range(1, len(GameTables)):
        TableScores = TableScores + GameTables[tableIdx].map(lambda x: [sum(x)])
    
    def findMode(values):                        
        k = int(1 + math.log2(len(GameTables)))
        
        minVal = min(values)
        maxVal = max(values)        
        if maxVal == minVal:
            return minVal
    
        Intervals = [[] for _ in range(k)]
        for val in values:    
            idx = int((val - minVal) / ((maxVal - minVal) / k))        
            if idx >= k:
                idx = k - 1
            Intervals[idx].append(val)        

        maxCount = 0
        modeIdx = 0
        for i, val in enumerate(Intervals):
            if len(val) > maxCount:
                maxCount = len(val)
                modeIdx = i
                        
        return sum(Intervals[modeIdx]) / len(Intervals[modeIdx])        
    
    TableMode = TableScores.map(findMode)
    return TableMode


TableAvg = CalcAvg(GameTables )
print("=====================AVG=========================")


print(TableAvg)



=====================AVG=========================
          Alex      Bob   Clara    Denis    Emma   Frida  George    Hawk    Ivan    Jack   Kevin   Lucas     Max   Natan
Alex    200.00  1000.00  204.00  1000.00  960.00  204.00  232.00  599.00  916.84  405.92  801.36  902.32  563.88  604.92
Bob       0.00   600.00  600.00     3.00  570.00  600.00   24.00  299.82  538.47  600.00  151.50  531.78  268.05  298.41
Clara   199.00   600.00  600.00   450.00  588.00  600.00  599.00  449.85  578.10  600.00  449.30  548.40  399.84  432.01
Denis     0.00   998.00  450.00   400.00  940.00    9.00   38.00  444.75  878.08  523.96  445.78  866.25  473.69  465.34
Emma     10.00   620.00  593.00    40.00  580.00   71.00   34.00  318.21  560.32  600.62  176.69  532.81  281.34  320.91
Frida   199.00   600.00  600.00   989.00  921.00  600.00  221.00  597.58  903.66  600.00  789.30  895.44  554.28  590.74
George  192.00   984.00  599.00   958.00  944.00  211.00  598.00  514.15  807.22  548.55  691.89  878.5

In [212]:
print("=====================DISP=========================")
pd.set_option('display.precision', 4)
print(CalcDisp(GameTables, TableAvg))

=====================DISP=========================
100
           Alex      Bob     Clara     Denis     Emma    Frida    George     Hawk      Ivan     Jack     Kevin     Lucas       Max    Natan
Alex     0.0000   0.0000    0.0000    0.0000   0.0000   0.0000    0.0000  32.5080   15.3536  24.2711   24.3889   63.2843  201.9368  58.6497
Bob      0.0000   0.0000    0.0000    0.0000   0.0000   0.0000    0.0000  22.2376   11.1151   0.0000   20.0204   42.4556  155.8375  37.2789
Clara    0.0000   0.0000    0.0000    0.0000   0.0000   0.0000    0.0000  15.7649    5.5859   0.0000    8.4393   32.1945  104.8108  31.5651
Denis    0.0000   0.0000    0.0000    0.0000   0.0000   0.0000    0.0000  33.2020   22.9116  13.7113   59.9913   70.0843  278.2647  62.6506
Emma     0.0000   0.0000    0.0000    0.0000   0.0000   0.0000    0.0000  25.4924   14.8173   3.9639   18.8849   37.2317  161.1623  41.7980
Frida    0.0000   0.0000    0.0000    0.0000   0.0000   0.0000    0.0000  26.5578   17.6362   0.0000   24

In [213]:
print("=====================MED=========================")
print(CalcМеdian(GameTables))


=====================MED=========================
        Alex   Bob  Clara  Denis  Emma  Frida  George  Hawk  Ivan  Jack  Kevin  Lucas  Max  Natan
Alex     200  1000    204   1000   960    204     232   620   888   420    828    800  900    664
Bob        0   600    600      3   570    600      24   312   531   600    159    456   72    369
Clara    199   600    600    450   588    600     599   454   580   600    442    577  537    489
Denis      0   998    450    400   940      9      38   408   864   551    444    943  148    464
Emma      10   620    593     40   580     71      34   328   541   599    159    593  295    376
Frida    199   600    600    989   921    600     221   619   917   600    799    851  343    507
George   192   984    599    958   944    211     598   459   889   599    460    872  965    533
Hawk     105   812    437    503   762    111     436   431   730   542    466    627  466    510
Ivan      29   642    573    113   618    100      52   320   583   

In [214]:
print("=====================MODE========================")
TableMode = CalcMode(GameTables)
print(TableMode)

=====================MODE========================
            Alex        Bob     Clara      Denis      Emma     Frida    George      Hawk      Ivan      Jack     Kevin     Lucas       Max     Natan
Alex    200.0000  1000.0000  204.0000  1000.0000  960.0000  204.0000  232.0000  612.8276  916.0000  413.0286  790.6087  983.8400  581.5135  620.3429
Bob       0.0000   600.0000  600.0000     3.0000  570.0000  600.0000   24.0000  301.0541  540.0000  600.0000  160.1250  585.3158  292.9714  309.9231
Clara   199.0000   600.0000  600.0000   450.0000  588.0000  600.0000  599.0000  459.5000  579.4348  600.0000  448.0833  507.0000  402.9355  426.4800
Denis     0.0000   998.0000  450.0000   400.0000  940.0000    9.0000   38.0000  456.3750  876.2000  530.5000  459.5385  936.6364  487.0294  478.7500
Emma     10.0000   620.0000  593.0000    40.0000  580.0000   71.0000   34.0000  309.9565  561.8182  602.0000  173.3846  500.2000  307.3824  303.2400
Frida   199.0000   600.0000  600.0000   989.0000  921.00